# SPensiones - v2

In [1]:
import requests
from datetime import datetime
import pandas as pd

# Para actualizar data SPensiones

In [ ]:
inityear = {'A': 2002, 'B': 2002, 'C': 1981, 'D': 2002, 'E': 2000}

today = datetime.now()
fin = today.year
conf = today.strftime("%Y%m%d")

for tf, ini in inityear.items():
    source = f'https://www.spensiones.cl/apps/valoresCuotaFondo/vcfAFPxls.php?aaaaini={ini}&aaaafin={fin}&tf={tf}&fecconf={conf}'
    r = requests.get(source)
    path = f'vcf{tf}{ini}-{fin}.csv'

    with open(path, 'wb') as out_file:
        out_file.write(r.content)

# Para leer data

In [8]:
dfA = pd.read_csv('vcfA2002-2022.csv', sep = ';', skiprows = 6384, index_col = False, decimal = ',', thousands = ".")
dfB = pd.read_csv('vcfB2002-2022.csv', sep = ';', skiprows = 6384, index_col = False, decimal = ',', thousands = ".")
dfC = pd.read_csv('vcfC1981-2022.csv', sep = ';', skiprows = 14345, index_col = False, decimal = ',', thousands = ".")
dfD = pd.read_csv('vcfD2002-2022.csv', sep = ';', skiprows = 6384, index_col = False, decimal = ',', thousands = ".")
dfE = pd.read_csv('vcfE2000-2022.csv', sep = ';', skiprows = 7282, index_col = False, decimal = ',', thousands = ".")

cols = pd.MultiIndex.from_tuples([("Fecha",""), ("CAPITAL","Valor Cuota"), ("CAPITAL","Valor Patrimonio"),
                                  ("CUPRUM","Valor Cuota"), ("CUPRUM","Valor Patrimonio"),
                                  ("HABITAT","Valor Cuota"), ("HABITAT","Valor Patrimonio"),
                                 ("MODELO","Valor Cuota"), ("MODELO","Valor Patrimonio"),
                                 ("PLANVITAL","Valor Cuota"), ("PLANVITAL","Valor Patrimonio"),
                                 ("PROVIDA","Valor Cuota"), ("PROVIDA","Valor Patrimonio"),
                                 ("UNO","Valor Cuota"), ("UNO","Valor Patrimonio")])
# set cols to df.columns
dfA.columns = cols
dfB.columns = cols
dfC.columns = cols
dfD.columns = cols
dfE.columns = cols

dfA['Fondo'] = 'Fondo A'
dfB['Fondo'] = 'Fondo B'
dfC['Fondo'] = 'Fondo C'
dfD['Fondo'] = 'Fondo D'
dfE['Fondo'] = 'Fondo E'
df = pd.concat([dfA, dfB, dfC, dfD, dfE])

In [11]:
df.tail()

Fecha     CAPITAL                       CUPRUM                   \
                 Valor Cuota Valor Patrimonio Valor Cuota Valor Patrimonio   
1025  2022-07-03   50.184,44    4723652976755   52.369,03    4416892257212   
1026  2022-07-04   50.406,62    4743163719541   52.594,84    4445645684928   
1027  2022-07-05   50.485,41    4757207226243   52.670,15    4445377866517   
1028  2022-07-06   50.733,89    4775821766412   52.906,55    4463693288108   
1029  2022-07-07   50.822,67    4781274851179   52.969,84    4467264690161   

         HABITAT                       MODELO                    PLANVITAL  \
     Valor Cuota Valor Patrimonio Valor Cuota Valor Patrimonio Valor Cuota   
1025   47.803,82    5843494575926   51.331,81    1302401971434   80.395,41   
1026   48.022,12    5886243661166   51.546,14    1312509447450   80.717,82   
1027   48.031,51    5897652468479   51.619,44    1316609545015   80.813,45   
1028   48.260,24    5920518940122   51.870,74    1324212182162   81.199,54   
1029   48.336,44    5934388612352   51.964,65    1326847505430   81.326,02   

                          PROVIDA                          UNO  \
     Valor Patrimonio Valor Cuota Valor Patrimonio Valor Cuota   
1025     789190441220   47.049,11    3613668314596   51.657,74   
1026     794011714853   47.246,59    3630930487624   51.866,55   
1027     795003963441   47.309,04    3640515662712   51.938,80   
1028     798747445754   47.541,22    3659021357176   52.164,27   
1029     799753515917   47.595,42    3663881034148   52.252,48   

                         Fondo  
     Valor Patrimonio           
1025     165317008965  Fondo E  
1026     166061327879  Fondo E  
1027     166312477145  Fondo E  
1028     167027232046  Fondo E  
1029     167525497208  Fondo E

# Calculos de Rentabilidades